# METHODOLOGY AND EXPERIMENTATION CPI FORECASTING

#### Import libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import zscore

In [2]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error, r2_score
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import matplotlib.pyplot as plt

#### Read File 

In [3]:
data = pd.read_csv(r"C:\Users\Owner\Downloads\MRP\CPI\CPI_Monthly_2003_2023_updated.csv", parse_dates=[0],index_col=[0]
                  )
data.head()


,All-items,Food,Shelter,Household operations,Clothing and footwear,Transportation,Gasoline,Health and personal care,Recreation & education,"Alcohol, tobacco & cannabis",All-items excluding food and energy,All-items excluding energy,Energy,Goods,Services
Date,,,,,,,,,,,,,,,
2003-01-01,102.0,101.6,102.1,100.1,96.6,105.5,108.4,100.5,99.3,106.8,101.5,101.5,108.3,101.9,102.1
2003-02-01,102.8,101.5,102.5,100.6,98.7,107.1,116.1,100.8,100.6,108.1,102.1,102.0,113.0,103.0,102.6
2003-03-01,103.1,101.7,103.2,100.6,100.3,107.6,117.2,100.5,100.3,108.3,102.3,102.2,115.3,103.6,102.7
2003-04-01,102.4,101.8,102.3,100.7,98.7,105.4,106.7,101.2,100.0,108.9,102.3,102.2,106.0,101.9,102.9
2003-05-01,102.5,102.1,102.9,100.6,98.3,104.0,99.3,101.4,100.7,110.2,102.5,102.4,103.7,101.8,103.2


#### Get Basic summary stastistics and information

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 244 entries, 2003-01-01 to 2023-04-01
Data columns (total 15 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   All-items                             244 non-null    float64
 1   Food                                  244 non-null    float64
 2   Shelter                               244 non-null    float64
 3   Household operations                  244 non-null    float64
 4   Clothing and footwear                 244 non-null    float64
 5   Transportation                        244 non-null    float64
 6   Gasoline                              244 non-null    float64
 7   Health and personal care              244 non-null    float64
 8   Recreation & education                244 non-null    float64
 9   Alcohol, tobacco & cannabis           244 non-null    float64
 10  All-items excluding food and energy   244 non-null    float64
 11  

In [5]:
data.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
All-items,244.0,123.229508,13.362860,102.0,112.175,122.70,133.000,156.4
Food,244.0,132.002459,20.085299,100.7,112.600,131.80,144.700,183.1
Shelter,244.0,129.881148,16.165007,102.1,119.500,127.95,140.075,169.9
Household operations,244.0,114.036885,9.665732,100.1,104.000,114.05,122.725,133.7
Clothing and footwear,244.0,94.535246,2.662912,87.9,92.775,94.65,96.225,100.8
Transportation,244.0,128.072541,15.869033,103.7,116.700,127.35,137.125,176.2
Gasoline,244.0,161.768852,33.503064,99.3,142.525,157.40,183.350,299.4
Health and personal care,244.0,117.973361,10.583401,100.5,107.900,118.40,125.600,145.7
Recreation & education,244.0,108.413115,7.160660,99.1,102.075,106.40,114.500,127.1
"Alcohol, tobacco & cannabis",244.0,144.764754,21.909026,106.8,126.600,139.60,166.475,189.4


In [6]:
#checking for missing values
data.isnull().sum()

All-items                               0
Food                                    0
Shelter                                 0
Household operations                    0
Clothing and footwear                   0
Transportation                          0
Gasoline                                0
Health and personal care                0
Recreation & education                  0
Alcohol, tobacco & cannabis             0
All-items excluding food and energy     0
All-items excluding energy              0
Energy                                  0
Goods                                   0
Services                                0
dtype: int64

In [7]:
#removing outliers
def remove_outliers_iqr(data, column_name):
    Q1 = data[column_name].quantile(0.25)
    Q3 = data[column_name].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    data_no_outliers = data[(data[column_name] >= lower_bound) & (data[column_name] <= upper_bound)]
    return data_no_outliers

# Remove outliers from the 'Gasoline' column
data = remove_outliers_iqr(data, 'Gasoline')

# Remove outliers from the 'Energy' column
data = remove_outliers_iqr(data, 'Energy')

# PROPOSED MODEL 

### RNN Encoder Decoder Single Head Attention Model with Teacher Forcing

In [8]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, LSTM, TimeDistributed, Activation, Dot, Concatenate
from tensorflow.keras.models import Model

# Spliting the data into training, validation, and test sets
train_data, test_data = train_test_split(data, test_size=0.2)
train_data, val_data = train_test_split(train_data, test_size=0.2)

# Defining the encoder and decoder inputs
encoder_inputs = Input(shape=(None, 1))
decoder_inputs = Input(shape=(None, 1))

# Defining the encoder LSTM layer
encoder_lstm = LSTM(180, return_sequences=True, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c]

# Defining the decoder LSTM layer
decoder_lstm = LSTM(180, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)

# Adding the Attention mechanism 
attention = Dot(axes=[2, 2])([decoder_outputs, encoder_outputs])
attention = Activation('softmax')(attention)

context = Dot(axes=[2, 1])([attention, encoder_outputs])
decoder_combined_context = Concatenate(axis=-1)([context, decoder_outputs])

# Output layer
output = TimeDistributed(Dense(1, activation='linear'))(decoder_combined_context)

# Defining the model
model = Model([encoder_inputs, decoder_inputs], output)
model.compile(optimizer='adam', loss='mse')

# Prepare data for teacher forcing
encoder_input_train = train_data.values[:-1, :, np.newaxis]
decoder_output_train = train_data.values[1:, :, np.newaxis]
decoder_input_train = np.zeros_like(decoder_output_train)  # Initializing  decoder inputs

# Teacher forcing: Setting up decoder inputs with true outputs from previous timestep
decoder_input_train[:-1] = decoder_output_train[:-1]

encoder_input_val = val_data.values[:-1, :, np.newaxis]
decoder_input_val = val_data.values[1:, :, np.newaxis]
decoder_output_val = val_data.values[1:, :, np.newaxis]

# Train the model using teacher forcing
model.fit(
    [encoder_input_train, decoder_input_train],
    decoder_output_train,
    validation_data=([encoder_input_val, decoder_input_val], decoder_output_val),
    epochs=500,
    batch_size=4
)

# Evaluate the model on the test set
encoder_input_test = test_data.values[:-1, :, np.newaxis]
decoder_input_test = test_data.values[1:, :, np.newaxis]
decoder_output_test = test_data.values[1:, :, np.newaxis]

preds = model.predict([encoder_input_test, decoder_input_test])
predictions = preds.squeeze()
test_values = decoder_output_test.squeeze()

mae = mean_absolute_error(test_values, predictions)
smape = 100 * np.mean(2 * np.abs(predictions - test_values) / (np.abs(predictions) + np.abs(test_values)))
rmse = np.sqrt(mean_squared_error(test_values, predictions))
r2 = r2_score(test_values, predictions)

print("MAE:", mae)
print("SMAPE:", smape)
print("RMSE:", rmse)
print("R2 Score:", r2)


Epoch 1/500
39/39 [==============================] - 4s 27ms/step - loss: 15193.3613 - val_loss: 11572.3330
Epoch 2/500
39/39 [==============================] - 0s 12ms/step - loss: 10723.6787 - val_loss: 8149.5757
Epoch 3/500
39/39 [==============================] - 0s 12ms/step - loss: 7782.9639 - val_loss: 5686.3276
Epoch 4/500
39/39 [==============================] - 0s 12ms/step - loss: 5473.3188 - val_loss: 3876.5564
Epoch 5/500
39/39 [==============================] - 0s 12ms/step - loss: 3856.8721 - val_loss: 2659.3240
Epoch 6/500
39/39 [==============================] - 0s 12ms/step - loss: 2741.3459 - val_loss: 1825.5607
Epoch 7/500
39/39 [==============================] - 0s 13ms/step - loss: 1976.4415 - val_loss: 1284.4763
Epoch 8/500
39/39 [==============================] - 1s 14ms/step - loss: 1464.4257 - val_loss: 939.0129
Epoch 9/500
39/39 [==============================] - 0s 12ms/step - loss: 1125.4308 - val_loss: 720.7023
Epoch 10/500
39/39 [=========================

Epoch 80/500
39/39 [==============================] - 1s 13ms/step - loss: 17.5754 - val_loss: 12.4764
Epoch 81/500
39/39 [==============================] - 1s 13ms/step - loss: 17.1977 - val_loss: 12.5894
Epoch 82/500
39/39 [==============================] - 1s 14ms/step - loss: 17.2179 - val_loss: 12.2934
Epoch 83/500
39/39 [==============================] - 1s 14ms/step - loss: 16.5926 - val_loss: 11.9479
Epoch 84/500
39/39 [==============================] - 1s 17ms/step - loss: 16.0970 - val_loss: 11.6004
Epoch 85/500
39/39 [==============================] - 1s 15ms/step - loss: 15.8175 - val_loss: 11.6654
Epoch 86/500
39/39 [==============================] - 1s 14ms/step - loss: 15.5050 - val_loss: 11.6544
Epoch 87/500
39/39 [==============================] - 1s 14ms/step - loss: 15.0695 - val_loss: 10.9041
Epoch 88/500
39/39 [==============================] - 1s 14ms/step - loss: 14.7878 - val_loss: 10.9399
Epoch 89/500
39/39 [==============================] - 0s 13ms/step - loss

39/39 [==============================] - 1s 13ms/step - loss: 2.8063 - val_loss: 2.0633
Epoch 161/500
39/39 [==============================] - 1s 13ms/step - loss: 2.8239 - val_loss: 2.0749
Epoch 162/500
39/39 [==============================] - 1s 14ms/step - loss: 3.1493 - val_loss: 2.1771
Epoch 163/500
39/39 [==============================] - 1s 14ms/step - loss: 2.8433 - val_loss: 2.3807
Epoch 164/500
39/39 [==============================] - 1s 14ms/step - loss: 2.7555 - val_loss: 1.9264
Epoch 165/500
39/39 [==============================] - 1s 13ms/step - loss: 2.6458 - val_loss: 1.8198
Epoch 166/500
39/39 [==============================] - 1s 14ms/step - loss: 2.5471 - val_loss: 1.9005
Epoch 167/500
39/39 [==============================] - 1s 13ms/step - loss: 2.5922 - val_loss: 1.8065
Epoch 168/500
39/39 [==============================] - 1s 13ms/step - loss: 2.5752 - val_loss: 1.8298
Epoch 169/500
39/39 [==============================] - 1s 13ms/step - loss: 2.4342 - val_loss: 1

39/39 [==============================] - 1s 13ms/step - loss: 0.3966 - val_loss: 0.6066
Epoch 321/500
39/39 [==============================] - 1s 13ms/step - loss: 0.3561 - val_loss: 0.1591
Epoch 322/500
39/39 [==============================] - 1s 14ms/step - loss: 0.2047 - val_loss: 0.5274
Epoch 323/500
39/39 [==============================] - 1s 14ms/step - loss: 0.1833 - val_loss: 0.2636
Epoch 324/500
39/39 [==============================] - 1s 16ms/step - loss: 0.1962 - val_loss: 0.2810
Epoch 325/500
39/39 [==============================] - 1s 15ms/step - loss: 0.1892 - val_loss: 0.3185
Epoch 326/500
39/39 [==============================] - 1s 14ms/step - loss: 0.2617 - val_loss: 0.6770
Epoch 327/500
39/39 [==============================] - 1s 13ms/step - loss: 0.8881 - val_loss: 1.5475
Epoch 328/500
39/39 [==============================] - 1s 13ms/step - loss: 2.9179 - val_loss: 0.8439
Epoch 329/500
39/39 [==============================] - 1s 13ms/step - loss: 1.4817 - val_loss: 0

39/39 [==============================] - 1s 14ms/step - loss: 0.0685 - val_loss: 0.0382
Epoch 481/500
39/39 [==============================] - 1s 14ms/step - loss: 0.1330 - val_loss: 0.1364
Epoch 482/500
39/39 [==============================] - 1s 13ms/step - loss: 0.1680 - val_loss: 0.3017
Epoch 483/500
39/39 [==============================] - 1s 14ms/step - loss: 0.3841 - val_loss: 0.6498
Epoch 484/500
39/39 [==============================] - 1s 13ms/step - loss: 0.2497 - val_loss: 0.0738
Epoch 485/500
39/39 [==============================] - 1s 14ms/step - loss: 0.0573 - val_loss: 0.0639
Epoch 486/500
39/39 [==============================] - 1s 14ms/step - loss: 0.0570 - val_loss: 0.1212
Epoch 487/500
39/39 [==============================] - 1s 15ms/step - loss: 0.0501 - val_loss: 0.0747
Epoch 488/500
39/39 [==============================] - 1s 15ms/step - loss: 0.0692 - val_loss: 0.0920
Epoch 489/500
39/39 [==============================] - 1s 15ms/step - loss: 0.1738 - val_loss: 0

## Testing Proposed Model w/TF with Rolling window of  36

In [8]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, LSTM, TimeDistributed, Activation, Dot, Concatenate
from tensorflow.keras.models import Model

# Spliting the data into training, validation, and test sets
train_data, test_data = train_test_split(data, test_size=0.2)
train_data, val_data = train_test_split(train_data, test_size=0.2)

# Define the encoder and decoder inputs
encoder_inputs = Input(shape=(None, 1))
decoder_inputs = Input(shape=(None, 1))

# Define the encoder LSTM layer
encoder_lstm = LSTM(180, return_sequences=True, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c]

# Define the decoder LSTM layer
decoder_lstm = LSTM(180, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)

# Adding the Attention mechanism 
attention = Dot(axes=[2, 2])([decoder_outputs, encoder_outputs])
attention = Activation('softmax')(attention)

context = Dot(axes=[2, 1])([attention, encoder_outputs])
decoder_combined_context = Concatenate(axis=-1)([context, decoder_outputs])

# Output layer
output = TimeDistributed(Dense(1, activation='linear'))(decoder_combined_context)

# Define the model
model = Model([encoder_inputs, decoder_inputs], output)
model.compile(optimizer='adam', loss='mse')

# Prepare data using the last 36 data points for the rolling window
window_size = 36

encoder_input_train = train_data.values[-window_size-1:-1, :, np.newaxis]
decoder_output_train = train_data.values[-window_size:, :, np.newaxis]
decoder_input_train = np.zeros_like(decoder_output_train)  # Initializing decoder inputs

# Teacher forcing: Setting up decoder inputs with true outputs from previous timestep
decoder_input_train[:-1] = decoder_output_train[:-1]

encoder_input_val = val_data.values[-window_size-1:-1, :, np.newaxis]
decoder_input_val = val_data.values[-window_size:, :, np.newaxis]
decoder_output_val = val_data.values[-window_size:, :, np.newaxis]

encoder_input_test = test_data.values[-window_size-1:-1, :, np.newaxis]
decoder_input_test = test_data.values[-window_size:, :, np.newaxis]
decoder_output_test = test_data.values[-window_size:, :, np.newaxis]

# Train the model using teacher forcing
model.fit(
    [encoder_input_train, decoder_input_train],
    decoder_output_train,
    validation_data=([encoder_input_val, decoder_input_val], decoder_output_val),
    epochs=500,
    batch_size=4
)

# Evaluate the model on the test set
preds = model.predict([encoder_input_test, decoder_input_test])
predictions = preds.squeeze()
test_values = decoder_output_test.squeeze()

mae = mean_absolute_error(test_values, predictions)
smape = 100 * np.mean(2 * np.abs(predictions - test_values) / (np.abs(predictions) + np.abs(test_values)))
rmse = np.sqrt(mean_squared_error(test_values, predictions))
r2 = r2_score(test_values, predictions)

print("MAE:", mae)
print("SMAPE:", smape)
print("RMSE:", rmse)
print("R2 Score:", r2)


Epoch 1/500
9/9 [==============================] - 4s 123ms/step - loss: 14537.5674 - val_loss: 15664.4531
Epoch 2/500
9/9 [==============================] - 0s 18ms/step - loss: 13805.7783 - val_loss: 14981.5488
Epoch 3/500
9/9 [==============================] - 0s 20ms/step - loss: 13031.9961 - val_loss: 14007.0361
Epoch 4/500
9/9 [==============================] - 0s 17ms/step - loss: 11990.5732 - val_loss: 12841.9805
Epoch 5/500
9/9 [==============================] - 0s 18ms/step - loss: 10903.9463 - val_loss: 11705.5371
Epoch 6/500
9/9 [==============================] - 0s 20ms/step - loss: 9898.7148 - val_loss: 10778.2666
Epoch 7/500
9/9 [==============================] - 0s 18ms/step - loss: 9166.9111 - val_loss: 10072.5693
Epoch 8/500
9/9 [==============================] - 0s 18ms/step - loss: 8497.1631 - val_loss: 9342.4102
Epoch 9/500
9/9 [==============================] - 0s 16ms/step - loss: 7823.1128 - val_loss: 8643.1543
Epoch 10/500
9/9 [==============================] -

Epoch 80/500
9/9 [==============================] - 0s 27ms/step - loss: 361.4783 - val_loss: 499.2924
Epoch 81/500
9/9 [==============================] - 0s 23ms/step - loss: 361.0911 - val_loss: 498.5841
Epoch 82/500
9/9 [==============================] - 0s 23ms/step - loss: 360.6901 - val_loss: 497.8804
Epoch 83/500
9/9 [==============================] - 0s 23ms/step - loss: 360.3570 - val_loss: 496.5608
Epoch 84/500
9/9 [==============================] - 0s 26ms/step - loss: 359.9185 - val_loss: 495.6477
Epoch 85/500
9/9 [==============================] - 0s 24ms/step - loss: 359.5571 - val_loss: 494.8942
Epoch 86/500
9/9 [==============================] - 0s 26ms/step - loss: 359.2963 - val_loss: 493.6799
Epoch 87/500
9/9 [==============================] - 0s 22ms/step - loss: 358.8142 - val_loss: 492.9626
Epoch 88/500
9/9 [==============================] - 0s 25ms/step - loss: 358.4133 - val_loss: 492.5100
Epoch 89/500
9/9 [==============================] - 0s 25ms/step - loss: 

Epoch 159/500
9/9 [==============================] - 0s 27ms/step - loss: 337.1681 - val_loss: 465.9205
Epoch 160/500
9/9 [==============================] - 0s 24ms/step - loss: 337.0658 - val_loss: 464.7621
Epoch 161/500
9/9 [==============================] - 0s 21ms/step - loss: 336.6034 - val_loss: 464.8698
Epoch 162/500
9/9 [==============================] - 0s 21ms/step - loss: 336.5102 - val_loss: 464.8062
Epoch 163/500
9/9 [==============================] - 0s 24ms/step - loss: 336.3687 - val_loss: 463.8401
Epoch 164/500
9/9 [==============================] - 0s 22ms/step - loss: 335.8602 - val_loss: 464.2954
Epoch 165/500
9/9 [==============================] - 0s 22ms/step - loss: 335.7286 - val_loss: 463.2987
Epoch 166/500
9/9 [==============================] - 0s 21ms/step - loss: 335.4373 - val_loss: 464.2143
Epoch 167/500
9/9 [==============================] - 0s 21ms/step - loss: 335.0998 - val_loss: 463.4153
Epoch 168/500
9/9 [==============================] - 0s 24ms/ste

Epoch 238/500
9/9 [==============================] - 0s 27ms/step - loss: 138.6001 - val_loss: 186.7475
Epoch 239/500
9/9 [==============================] - 0s 21ms/step - loss: 137.3401 - val_loss: 173.8170
Epoch 240/500
9/9 [==============================] - 0s 20ms/step - loss: 134.3318 - val_loss: 183.4373
Epoch 241/500
9/9 [==============================] - 0s 19ms/step - loss: 120.9037 - val_loss: 173.8679
Epoch 242/500
9/9 [==============================] - 0s 23ms/step - loss: 118.5167 - val_loss: 167.4644
Epoch 243/500
9/9 [==============================] - 0s 20ms/step - loss: 113.3227 - val_loss: 162.3547
Epoch 244/500
9/9 [==============================] - 0s 21ms/step - loss: 110.3264 - val_loss: 157.8403
Epoch 245/500
9/9 [==============================] - 0s 26ms/step - loss: 108.0782 - val_loss: 156.6033
Epoch 246/500
9/9 [==============================] - 0s 20ms/step - loss: 107.0516 - val_loss: 152.8843
Epoch 247/500
9/9 [==============================] - 0s 20ms/ste

Epoch 318/500
9/9 [==============================] - 0s 25ms/step - loss: 43.3078 - val_loss: 59.1630
Epoch 319/500
9/9 [==============================] - 0s 23ms/step - loss: 43.0632 - val_loss: 58.3888
Epoch 320/500
9/9 [==============================] - 0s 21ms/step - loss: 43.7200 - val_loss: 56.9488
Epoch 321/500
9/9 [==============================] - 0s 23ms/step - loss: 42.5547 - val_loss: 56.9519
Epoch 322/500
9/9 [==============================] - 0s 25ms/step - loss: 42.5649 - val_loss: 57.2430
Epoch 323/500
9/9 [==============================] - 0s 21ms/step - loss: 44.3412 - val_loss: 56.5157
Epoch 324/500
9/9 [==============================] - 0s 24ms/step - loss: 46.1100 - val_loss: 53.9785
Epoch 325/500
9/9 [==============================] - 0s 22ms/step - loss: 45.1477 - val_loss: 53.9582
Epoch 326/500
9/9 [==============================] - 0s 21ms/step - loss: 43.8439 - val_loss: 56.2714
Epoch 327/500
9/9 [==============================] - 0s 22ms/step - loss: 44.3363 

9/9 [==============================] - 0s 24ms/step - loss: 25.7138 - val_loss: 26.4841
Epoch 399/500
9/9 [==============================] - 0s 24ms/step - loss: 24.3592 - val_loss: 25.4521
Epoch 400/500
9/9 [==============================] - 0s 23ms/step - loss: 23.5760 - val_loss: 25.7413
Epoch 401/500
9/9 [==============================] - 0s 23ms/step - loss: 24.0308 - val_loss: 26.2062
Epoch 402/500
9/9 [==============================] - 0s 24ms/step - loss: 27.0431 - val_loss: 25.2832
Epoch 403/500
9/9 [==============================] - 0s 25ms/step - loss: 26.2895 - val_loss: 25.4272
Epoch 404/500
9/9 [==============================] - 0s 23ms/step - loss: 23.2737 - val_loss: 25.3869
Epoch 405/500
9/9 [==============================] - 0s 23ms/step - loss: 24.8366 - val_loss: 25.3989
Epoch 406/500
9/9 [==============================] - 0s 27ms/step - loss: 22.8150 - val_loss: 24.5504
Epoch 407/500
9/9 [==============================] - 0s 22ms/step - loss: 25.5637 - val_loss: 25

9/9 [==============================] - 0s 23ms/step - loss: 12.0959 - val_loss: 10.5082
Epoch 479/500
9/9 [==============================] - 0s 25ms/step - loss: 10.8151 - val_loss: 9.4179
Epoch 480/500
9/9 [==============================] - 0s 23ms/step - loss: 14.9020 - val_loss: 16.4785
Epoch 481/500
9/9 [==============================] - 0s 24ms/step - loss: 14.6089 - val_loss: 10.2770
Epoch 482/500
9/9 [==============================] - 0s 22ms/step - loss: 14.8196 - val_loss: 9.5598
Epoch 483/500
9/9 [==============================] - 0s 25ms/step - loss: 12.3183 - val_loss: 9.6030
Epoch 484/500
9/9 [==============================] - 0s 24ms/step - loss: 15.7922 - val_loss: 32.1884
Epoch 485/500
9/9 [==============================] - 0s 24ms/step - loss: 21.4696 - val_loss: 43.6788
Epoch 486/500
9/9 [==============================] - 0s 21ms/step - loss: 18.3678 - val_loss: 12.8005
Epoch 487/500
9/9 [==============================] - 0s 20ms/step - loss: 16.3821 - val_loss: 15.63

## Testing Proposed Model w/TF with Rolling window of  24

In [9]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, LSTM, TimeDistributed, Activation, Dot, Concatenate
from tensorflow.keras.models import Model


# Spliting the data into training, validation, and test sets
train_data, test_data = train_test_split(data, test_size=0.2)
train_data, val_data = train_test_split(train_data, test_size=0.2)

# Define the encoder and decoder inputs
encoder_inputs = Input(shape=(None, 1))
decoder_inputs = Input(shape=(None, 1))

# Define the encoder LSTM layer
encoder_lstm = LSTM(180, return_sequences=True, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c]

# Define the decoder LSTM layer
decoder_lstm = LSTM(180, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)

# Adding the Attention mechanism 
attention = Dot(axes=[2, 2])([decoder_outputs, encoder_outputs])
attention = Activation('softmax')(attention)

context = Dot(axes=[2, 1])([attention, encoder_outputs])
decoder_combined_context = Concatenate(axis=-1)([context, decoder_outputs])

# Output layer
output = TimeDistributed(Dense(1, activation='linear'))(decoder_combined_context)

# Define the model
model = Model([encoder_inputs, decoder_inputs], output)
model.compile(optimizer='adam', loss='mse')

# Prepare data using the last 24 data points for the rolling window
window_size = 24

encoder_input_train = train_data.values[-window_size-1:-1, :, np.newaxis]
decoder_output_train = train_data.values[-window_size:, :, np.newaxis]
decoder_input_train = np.zeros_like(decoder_output_train)  # Initializing decoder inputs

# Teacher forcing: Setting up decoder inputs with true outputs from previous timestep
decoder_input_train[:-1] = decoder_output_train[:-1]

encoder_input_val = val_data.values[-window_size-1:-1, :, np.newaxis]
decoder_input_val = val_data.values[-window_size:, :, np.newaxis]
decoder_output_val = val_data.values[-window_size:, :, np.newaxis]

encoder_input_test = test_data.values[-window_size-1:-1, :, np.newaxis]
decoder_input_test = test_data.values[-window_size:, :, np.newaxis]
decoder_output_test = test_data.values[-window_size:, :, np.newaxis]

# Train the model using teacher forcing
model.fit(
    [encoder_input_train, decoder_input_train],
    decoder_output_train,
    validation_data=([encoder_input_val, decoder_input_val], decoder_output_val),
    epochs=500,
    batch_size=4
)

# Evaluate the model on the test set
preds = model.predict([encoder_input_test, decoder_input_test])
predictions = preds.squeeze()
test_values = decoder_output_test.squeeze()

mae = mean_absolute_error(test_values, predictions)
smape = 100 * np.mean(2 * np.abs(predictions - test_values) / (np.abs(predictions) + np.abs(test_values)))
rmse = np.sqrt(mean_squared_error(test_values, predictions))
r2 = r2_score(test_values, predictions)

print("MAE:", mae)
print("SMAPE:", smape)
print("RMSE:", rmse)
print("R2 Score:", r2)


Epoch 1/500
6/6 [==============================] - 4s 173ms/step - loss: 15885.2158 - val_loss: 15389.9814
Epoch 2/500
6/6 [==============================] - 0s 19ms/step - loss: 15415.0938 - val_loss: 14946.1299
Epoch 3/500
6/6 [==============================] - 0s 19ms/step - loss: 14961.0869 - val_loss: 14473.5029
Epoch 4/500
6/6 [==============================] - 0s 21ms/step - loss: 14472.9424 - val_loss: 13998.4766
Epoch 5/500
6/6 [==============================] - 0s 21ms/step - loss: 13970.3115 - val_loss: 13436.9482
Epoch 6/500
6/6 [==============================] - 0s 21ms/step - loss: 13341.3232 - val_loss: 12739.0039
Epoch 7/500
6/6 [==============================] - 0s 19ms/step - loss: 12612.4912 - val_loss: 12040.2139
Epoch 8/500
6/6 [==============================] - 0s 19ms/step - loss: 11937.7217 - val_loss: 11386.4854
Epoch 9/500
6/6 [==============================] - 0s 20ms/step - loss: 11284.5938 - val_loss: 10765.8535
Epoch 10/500
6/6 [===========================

6/6 [==============================] - 0s 19ms/step - loss: 512.1503 - val_loss: 511.8359
Epoch 80/500
6/6 [==============================] - 0s 18ms/step - loss: 504.8594 - val_loss: 506.4396
Epoch 81/500
6/6 [==============================] - 0s 19ms/step - loss: 499.0117 - val_loss: 501.3316
Epoch 82/500
6/6 [==============================] - 0s 18ms/step - loss: 493.6909 - val_loss: 496.4959
Epoch 83/500
6/6 [==============================] - 0s 23ms/step - loss: 488.7753 - val_loss: 491.9887
Epoch 84/500
6/6 [==============================] - 0s 18ms/step - loss: 483.5593 - val_loss: 488.0793
Epoch 85/500
6/6 [==============================] - 0s 21ms/step - loss: 479.7832 - val_loss: 484.3220
Epoch 86/500
6/6 [==============================] - 0s 18ms/step - loss: 475.2044 - val_loss: 481.1628
Epoch 87/500
6/6 [==============================] - 0s 19ms/step - loss: 471.5077 - val_loss: 478.3473
Epoch 88/500
6/6 [==============================] - 0s 18ms/step - loss: 468.2712 - va

6/6 [==============================] - 0s 30ms/step - loss: 431.9107 - val_loss: 450.5612
Epoch 159/500
6/6 [==============================] - 0s 30ms/step - loss: 431.7451 - val_loss: 450.4558
Epoch 160/500
6/6 [==============================] - 0s 27ms/step - loss: 431.7443 - val_loss: 450.4191
Epoch 161/500
6/6 [==============================] - 0s 27ms/step - loss: 431.7317 - val_loss: 450.2010
Epoch 162/500
6/6 [==============================] - 0s 24ms/step - loss: 431.4500 - val_loss: 450.1223
Epoch 163/500
6/6 [==============================] - 0s 25ms/step - loss: 431.3938 - val_loss: 449.9988
Epoch 164/500
6/6 [==============================] - 0s 23ms/step - loss: 431.2638 - val_loss: 449.8728
Epoch 165/500
6/6 [==============================] - 0s 25ms/step - loss: 431.1354 - val_loss: 449.7787
Epoch 166/500
6/6 [==============================] - 0s 29ms/step - loss: 431.1002 - val_loss: 449.7180
Epoch 167/500
6/6 [==============================] - 0s 24ms/step - loss: 431.

Epoch 237/500
6/6 [==============================] - 0s 23ms/step - loss: 422.4508 - val_loss: 439.8685
Epoch 238/500
6/6 [==============================] - 0s 24ms/step - loss: 421.8055 - val_loss: 438.9493
Epoch 239/500
6/6 [==============================] - 0s 23ms/step - loss: 421.0394 - val_loss: 438.4114
Epoch 240/500
6/6 [==============================] - 0s 23ms/step - loss: 420.4164 - val_loss: 437.5924
Epoch 241/500
6/6 [==============================] - 0s 24ms/step - loss: 419.3920 - val_loss: 435.9545
Epoch 242/500
6/6 [==============================] - 0s 27ms/step - loss: 418.6826 - val_loss: 435.1060
Epoch 243/500
6/6 [==============================] - 0s 26ms/step - loss: 417.6183 - val_loss: 434.2570
Epoch 244/500
6/6 [==============================] - 0s 23ms/step - loss: 416.6212 - val_loss: 432.6689
Epoch 245/500
6/6 [==============================] - 0s 24ms/step - loss: 415.4106 - val_loss: 431.4109
Epoch 246/500
6/6 [==============================] - 0s 24ms/ste

Epoch 316/500
6/6 [==============================] - 0s 26ms/step - loss: 156.8207 - val_loss: 142.9020
Epoch 317/500
6/6 [==============================] - 0s 23ms/step - loss: 151.8590 - val_loss: 138.9809
Epoch 318/500
6/6 [==============================] - 0s 23ms/step - loss: 148.1257 - val_loss: 134.6390
Epoch 319/500
6/6 [==============================] - 0s 25ms/step - loss: 144.0826 - val_loss: 133.1092
Epoch 320/500
6/6 [==============================] - 0s 23ms/step - loss: 140.5637 - val_loss: 126.7041
Epoch 321/500
6/6 [==============================] - 0s 26ms/step - loss: 138.6269 - val_loss: 128.6180
Epoch 322/500
6/6 [==============================] - 0s 25ms/step - loss: 138.9446 - val_loss: 125.2393
Epoch 323/500
6/6 [==============================] - 0s 23ms/step - loss: 133.9482 - val_loss: 123.8968
Epoch 324/500
6/6 [==============================] - 0s 24ms/step - loss: 134.2770 - val_loss: 118.3674
Epoch 325/500
6/6 [==============================] - 0s 28ms/ste

Epoch 396/500
6/6 [==============================] - 0s 22ms/step - loss: 61.8285 - val_loss: 45.1986
Epoch 397/500
6/6 [==============================] - 0s 24ms/step - loss: 63.1696 - val_loss: 45.0008
Epoch 398/500
6/6 [==============================] - 0s 25ms/step - loss: 61.8007 - val_loss: 44.4014
Epoch 399/500
6/6 [==============================] - 0s 25ms/step - loss: 60.4924 - val_loss: 43.0073
Epoch 400/500
6/6 [==============================] - 0s 21ms/step - loss: 59.8107 - val_loss: 42.1283
Epoch 401/500
6/6 [==============================] - 0s 23ms/step - loss: 58.3695 - val_loss: 42.0441
Epoch 402/500
6/6 [==============================] - 0s 25ms/step - loss: 59.3217 - val_loss: 44.0818
Epoch 403/500
6/6 [==============================] - 0s 24ms/step - loss: 59.4191 - val_loss: 42.0951
Epoch 404/500
6/6 [==============================] - 0s 22ms/step - loss: 57.6877 - val_loss: 44.4020
Epoch 405/500
6/6 [==============================] - 0s 26ms/step - loss: 56.9235 

6/6 [==============================] - 0s 25ms/step - loss: 33.2262 - val_loss: 18.8314
Epoch 477/500
6/6 [==============================] - 0s 22ms/step - loss: 32.8641 - val_loss: 19.0881
Epoch 478/500
6/6 [==============================] - 0s 26ms/step - loss: 34.5907 - val_loss: 17.9584
Epoch 479/500
6/6 [==============================] - 0s 23ms/step - loss: 37.9159 - val_loss: 21.4960
Epoch 480/500
6/6 [==============================] - 0s 22ms/step - loss: 36.6536 - val_loss: 19.2035
Epoch 481/500
6/6 [==============================] - 0s 23ms/step - loss: 33.6376 - val_loss: 20.3215
Epoch 482/500
6/6 [==============================] - 0s 24ms/step - loss: 33.7835 - val_loss: 19.4961
Epoch 483/500
6/6 [==============================] - 0s 26ms/step - loss: 32.6375 - val_loss: 19.0085
Epoch 484/500
6/6 [==============================] - 0s 27ms/step - loss: 31.3572 - val_loss: 17.7186
Epoch 485/500
6/6 [==============================] - 0s 27ms/step - loss: 31.8677 - val_loss: 17

## Testing Proposed Model w/TF with Rolling window of  12

In [10]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, LSTM, TimeDistributed, Activation, Dot, Concatenate
from tensorflow.keras.models import Model

# Spliting the data into training, validation, and test sets
train_data, test_data = train_test_split(data, test_size=0.2)
train_data, val_data = train_test_split(train_data, test_size=0.2)

# Define the encoder and decoder inputs
encoder_inputs = Input(shape=(None, 1))
decoder_inputs = Input(shape=(None, 1))

# Define the encoder LSTM layer
encoder_lstm = LSTM(180, return_sequences=True, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c]

# Define the decoder LSTM layer
decoder_lstm = LSTM(180, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)

# Adding the Attention mechanism 
attention = Dot(axes=[2, 2])([decoder_outputs, encoder_outputs])
attention = Activation('softmax')(attention)

context = Dot(axes=[2, 1])([attention, encoder_outputs])
decoder_combined_context = Concatenate(axis=-1)([context, decoder_outputs])

# Output layer
output = TimeDistributed(Dense(1, activation='linear'))(decoder_combined_context)

# Define the model
model = Model([encoder_inputs, decoder_inputs], output)
model.compile(optimizer='adam', loss='mse')

# Prepare data using the last 12 data points for the rolling window
window_size = 12

encoder_input_train = train_data.values[-window_size-1:-1, :, np.newaxis]
decoder_output_train = train_data.values[-window_size:, :, np.newaxis]
decoder_input_train = np.zeros_like(decoder_output_train)  # Initializing decoder inputs

# Teacher forcing: Setting up decoder inputs with true outputs from previous timestep
decoder_input_train[:-1] = decoder_output_train[:-1]

encoder_input_val = val_data.values[-window_size-1:-1, :, np.newaxis]
decoder_input_val = val_data.values[-window_size:, :, np.newaxis]
decoder_output_val = val_data.values[-window_size:, :, np.newaxis]

encoder_input_test = test_data.values[-window_size-1:-1, :, np.newaxis]
decoder_input_test = test_data.values[-window_size:, :, np.newaxis]
decoder_output_test = test_data.values[-window_size:, :, np.newaxis]

# Train the model using teacher forcing
model.fit(
    [encoder_input_train, decoder_input_train],
    decoder_output_train,
    validation_data=([encoder_input_val, decoder_input_val], decoder_output_val),
    epochs=500,
    batch_size=4
)

# Evaluate the model on the test set
preds = model.predict([encoder_input_test, decoder_input_test])
predictions = preds.squeeze()
test_values = decoder_output_test.squeeze()

mae = mean_absolute_error(test_values, predictions)
smape = 100 * np.mean(2 * np.abs(predictions - test_values) / (np.abs(predictions) + np.abs(test_values)))
rmse = np.sqrt(mean_squared_error(test_values, predictions))
r2 = r2_score(test_values, predictions)

print("MAE:", mae)
print("SMAPE:", smape)
print("RMSE:", rmse)
print("R2 Score:", r2)


Epoch 1/500
3/3 [==============================] - 3s 409ms/step - loss: 15978.7354 - val_loss: 14941.4307
Epoch 2/500
3/3 [==============================] - 0s 30ms/step - loss: 15654.6436 - val_loss: 14639.1143
Epoch 3/500
3/3 [==============================] - 0s 27ms/step - loss: 15363.1602 - val_loss: 14347.6758
Epoch 4/500
3/3 [==============================] - 0s 30ms/step - loss: 15070.8643 - val_loss: 14084.4619
Epoch 5/500
3/3 [==============================] - 0s 29ms/step - loss: 14798.4639 - val_loss: 13823.5547
Epoch 6/500
3/3 [==============================] - 0s 28ms/step - loss: 14527.1260 - val_loss: 13551.1992
Epoch 7/500
3/3 [==============================] - 0s 29ms/step - loss: 14231.2764 - val_loss: 13258.3955
Epoch 8/500
3/3 [==============================] - 0s 28ms/step - loss: 13934.9531 - val_loss: 12995.7607
Epoch 9/500
3/3 [==============================] - 0s 26ms/step - loss: 13658.5732 - val_loss: 12715.5781
Epoch 10/500
3/3 [===========================

Epoch 79/500
3/3 [==============================] - 0s 25ms/step - loss: 1711.9332 - val_loss: 1358.9508
Epoch 80/500
3/3 [==============================] - 0s 30ms/step - loss: 1664.8051 - val_loss: 1319.0958
Epoch 81/500
3/3 [==============================] - 0s 29ms/step - loss: 1623.5907 - val_loss: 1280.0010
Epoch 82/500
3/3 [==============================] - 0s 30ms/step - loss: 1583.4185 - val_loss: 1241.7832
Epoch 83/500
3/3 [==============================] - 0s 37ms/step - loss: 1542.5062 - val_loss: 1204.9398
Epoch 84/500
3/3 [==============================] - 0s 28ms/step - loss: 1500.3583 - val_loss: 1170.0547
Epoch 85/500
3/3 [==============================] - 0s 32ms/step - loss: 1463.7694 - val_loss: 1136.0084
Epoch 86/500
3/3 [==============================] - 0s 36ms/step - loss: 1423.9862 - val_loss: 1103.8883
Epoch 87/500
3/3 [==============================] - 0s 29ms/step - loss: 1389.3995 - val_loss: 1072.6237
Epoch 88/500
3/3 [==============================] - 0s 

3/3 [==============================] - 0s 34ms/step - loss: 503.2665 - val_loss: 343.5350
Epoch 158/500
3/3 [==============================] - 0s 33ms/step - loss: 500.5404 - val_loss: 342.4274
Epoch 159/500
3/3 [==============================] - 0s 32ms/step - loss: 498.4851 - val_loss: 341.3600
Epoch 160/500
3/3 [==============================] - 0s 28ms/step - loss: 496.4935 - val_loss: 340.3673
Epoch 161/500
3/3 [==============================] - 0s 29ms/step - loss: 495.0198 - val_loss: 339.4011
Epoch 162/500
3/3 [==============================] - 0s 29ms/step - loss: 493.0553 - val_loss: 338.5733
Epoch 163/500
3/3 [==============================] - 0s 32ms/step - loss: 491.2458 - val_loss: 337.7695
Epoch 164/500
3/3 [==============================] - 0s 30ms/step - loss: 489.9694 - val_loss: 336.9323
Epoch 165/500
3/3 [==============================] - 0s 33ms/step - loss: 488.1815 - val_loss: 336.1444
Epoch 166/500
3/3 [==============================] - 0s 34ms/step - loss: 486.

Epoch 236/500
3/3 [==============================] - 0s 26ms/step - loss: 333.0171 - val_loss: 245.2167
Epoch 237/500
3/3 [==============================] - 0s 28ms/step - loss: 330.6128 - val_loss: 243.0941
Epoch 238/500
3/3 [==============================] - 0s 28ms/step - loss: 328.1690 - val_loss: 240.8527
Epoch 239/500
3/3 [==============================] - 0s 24ms/step - loss: 325.4153 - val_loss: 237.6535
Epoch 240/500
3/3 [==============================] - 0s 28ms/step - loss: 323.9552 - val_loss: 239.6111
Epoch 241/500
3/3 [==============================] - 0s 31ms/step - loss: 320.6991 - val_loss: 235.4468
Epoch 242/500
3/3 [==============================] - 0s 41ms/step - loss: 319.1351 - val_loss: 235.8762
Epoch 243/500
3/3 [==============================] - 0s 30ms/step - loss: 317.5212 - val_loss: 234.2952
Epoch 244/500
3/3 [==============================] - 0s 30ms/step - loss: 315.8540 - val_loss: 233.6730
Epoch 245/500
3/3 [==============================] - 0s 30ms/ste

Epoch 315/500
3/3 [==============================] - 0s 35ms/step - loss: 148.5787 - val_loss: 95.7719
Epoch 316/500
3/3 [==============================] - 0s 37ms/step - loss: 147.4341 - val_loss: 95.1034
Epoch 317/500
3/3 [==============================] - 0s 43ms/step - loss: 145.9058 - val_loss: 93.6000
Epoch 318/500
3/3 [==============================] - 0s 34ms/step - loss: 145.2407 - val_loss: 92.9608
Epoch 319/500
3/3 [==============================] - 0s 33ms/step - loss: 143.1478 - val_loss: 91.8323
Epoch 320/500
3/3 [==============================] - 0s 33ms/step - loss: 141.7258 - val_loss: 92.0138
Epoch 321/500
3/3 [==============================] - 0s 35ms/step - loss: 140.7435 - val_loss: 90.9276
Epoch 322/500
3/3 [==============================] - 0s 39ms/step - loss: 139.5564 - val_loss: 90.4601
Epoch 323/500
3/3 [==============================] - 0s 37ms/step - loss: 138.2802 - val_loss: 90.3747
Epoch 324/500
3/3 [==============================] - 0s 33ms/step - loss:

Epoch 395/500
3/3 [==============================] - 0s 41ms/step - loss: 81.1012 - val_loss: 56.4550
Epoch 396/500
3/3 [==============================] - 0s 40ms/step - loss: 80.6976 - val_loss: 56.0890
Epoch 397/500
3/3 [==============================] - 0s 42ms/step - loss: 80.3870 - val_loss: 57.4736
Epoch 398/500
3/3 [==============================] - 0s 43ms/step - loss: 79.4395 - val_loss: 57.1583
Epoch 399/500
3/3 [==============================] - 0s 40ms/step - loss: 79.0415 - val_loss: 55.8520
Epoch 400/500
3/3 [==============================] - 0s 38ms/step - loss: 78.8272 - val_loss: 55.6569
Epoch 401/500
3/3 [==============================] - 0s 36ms/step - loss: 78.3752 - val_loss: 57.8663
Epoch 402/500
3/3 [==============================] - 0s 40ms/step - loss: 78.8377 - val_loss: 57.6369
Epoch 403/500
3/3 [==============================] - 0s 35ms/step - loss: 77.1695 - val_loss: 54.3515
Epoch 404/500
3/3 [==============================] - 0s 35ms/step - loss: 76.6399 

3/3 [==============================] - 0s 41ms/step - loss: 51.6996 - val_loss: 32.1073
Epoch 476/500
3/3 [==============================] - 0s 44ms/step - loss: 50.8030 - val_loss: 34.2164
Epoch 477/500
3/3 [==============================] - 0s 39ms/step - loss: 50.9128 - val_loss: 32.7506
Epoch 478/500
3/3 [==============================] - 0s 33ms/step - loss: 50.6976 - val_loss: 35.1445
Epoch 479/500
3/3 [==============================] - 0s 47ms/step - loss: 50.8403 - val_loss: 34.6796
Epoch 480/500
3/3 [==============================] - 0s 39ms/step - loss: 50.9659 - val_loss: 32.4744
Epoch 481/500
3/3 [==============================] - 0s 38ms/step - loss: 49.9163 - val_loss: 34.0496
Epoch 482/500
3/3 [==============================] - 0s 41ms/step - loss: 49.8516 - val_loss: 34.3285
Epoch 483/500
3/3 [==============================] - 0s 35ms/step - loss: 49.7548 - val_loss: 32.2514
Epoch 484/500
3/3 [==============================] - 0s 30ms/step - loss: 49.2653 - val_loss: 30

## Testing Proposed Model w/TF with Rolling window of  6

In [11]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, LSTM, TimeDistributed, Activation, Dot, Concatenate
from tensorflow.keras.models import Model


# Spliting the data into training, validation, and test sets
train_data, test_data = train_test_split(data, test_size=0.2)
train_data, val_data = train_test_split(train_data, test_size=0.2)

# Define the encoder and decoder inputs
encoder_inputs = Input(shape=(None, 1))
decoder_inputs = Input(shape=(None, 1))

# Define the encoder LSTM layer
encoder_lstm = LSTM(180, return_sequences=True, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c]

# Define the decoder LSTM layer
decoder_lstm = LSTM(180, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)

# Adding the Attention mechanism 
attention = Dot(axes=[2, 2])([decoder_outputs, encoder_outputs])
attention = Activation('softmax')(attention)

context = Dot(axes=[2, 1])([attention, encoder_outputs])
decoder_combined_context = Concatenate(axis=-1)([context, decoder_outputs])

# Output layer
output = TimeDistributed(Dense(1, activation='linear'))(decoder_combined_context)

# Define the model
model = Model([encoder_inputs, decoder_inputs], output)
model.compile(optimizer='adam', loss='mse')

# Prepare data using the last 6 data points for the rolling window
window_size = 6

encoder_input_train = train_data.values[-window_size-1:-1, :, np.newaxis]
decoder_output_train = train_data.values[-window_size:, :, np.newaxis]
decoder_input_train = np.zeros_like(decoder_output_train)  # Initializing decoder inputs

# Teacher forcing: Setting up decoder inputs with true outputs from previous timestep
decoder_input_train[:-1] = decoder_output_train[:-1]

encoder_input_val = val_data.values[-window_size-1:-1, :, np.newaxis]
decoder_input_val = val_data.values[-window_size:, :, np.newaxis]
decoder_output_val = val_data.values[-window_size:, :, np.newaxis]

encoder_input_test = test_data.values[-window_size-1:-1, :, np.newaxis]
decoder_input_test = test_data.values[-window_size:, :, np.newaxis]
decoder_output_test = test_data.values[-window_size:, :, np.newaxis]

# Train the model using teacher forcing
model.fit(
    [encoder_input_train, decoder_input_train],
    decoder_output_train,
    validation_data=([encoder_input_val, decoder_input_val], decoder_output_val),
    epochs=500,
    batch_size=4
)

# Evaluate the model on the test set
preds = model.predict([encoder_input_test, decoder_input_test])
predictions = preds.squeeze()
test_values = decoder_output_test.squeeze()

mae = mean_absolute_error(test_values, predictions)
smape = 100 * np.mean(2 * np.abs(predictions - test_values) / (np.abs(predictions) + np.abs(test_values)))
rmse = np.sqrt(mean_squared_error(test_values, predictions))
r2 = r2_score(test_values, predictions)

print("MAE:", mae)
print("SMAPE:", smape)
print("RMSE:", rmse)
print("R2 Score:", r2)


Epoch 1/500
2/2 [==============================] - 4s 776ms/step - loss: 13359.4414 - val_loss: 13991.7764
Epoch 2/500
2/2 [==============================] - 0s 45ms/step - loss: 13176.0508 - val_loss: 13810.8311
Epoch 3/500
2/2 [==============================] - 0s 48ms/step - loss: 12990.9600 - val_loss: 13630.8623
Epoch 4/500
2/2 [==============================] - 0s 45ms/step - loss: 12807.4482 - val_loss: 13457.3818
Epoch 5/500
2/2 [==============================] - 0s 45ms/step - loss: 12630.9170 - val_loss: 13287.1016
Epoch 6/500
2/2 [==============================] - 0s 53ms/step - loss: 12448.3350 - val_loss: 13116.4502
Epoch 7/500
2/2 [==============================] - 0s 50ms/step - loss: 12270.2803 - val_loss: 12943.5000
Epoch 8/500
2/2 [==============================] - 0s 53ms/step - loss: 12088.2764 - val_loss: 12765.1729
Epoch 9/500
2/2 [==============================] - 0s 46ms/step - loss: 11899.7100 - val_loss: 12576.4561
Epoch 10/500
2/2 [===========================

Epoch 79/500
2/2 [==============================] - 0s 52ms/step - loss: 2427.7605 - val_loss: 2747.3928
Epoch 80/500
2/2 [==============================] - 0s 58ms/step - loss: 2372.2803 - val_loss: 2688.0671
Epoch 81/500
2/2 [==============================] - 0s 51ms/step - loss: 2319.8206 - val_loss: 2630.2031
Epoch 82/500
2/2 [==============================] - 0s 53ms/step - loss: 2267.2356 - val_loss: 2574.2571
Epoch 83/500
2/2 [==============================] - 0s 49ms/step - loss: 2213.4602 - val_loss: 2520.7910
Epoch 84/500
2/2 [==============================] - 0s 57ms/step - loss: 2164.3191 - val_loss: 2468.8538
Epoch 85/500
2/2 [==============================] - 0s 49ms/step - loss: 2115.9292 - val_loss: 2418.6003
Epoch 86/500
2/2 [==============================] - 0s 49ms/step - loss: 2067.4082 - val_loss: 2369.8271
Epoch 87/500
2/2 [==============================] - 0s 51ms/step - loss: 2022.9772 - val_loss: 2321.8469
Epoch 88/500
2/2 [==============================] - 0s 

2/2 [==============================] - 0s 48ms/step - loss: 492.7901 - val_loss: 670.9039
Epoch 158/500
2/2 [==============================] - 0s 47ms/step - loss: 484.9074 - val_loss: 662.4153
Epoch 159/500
2/2 [==============================] - 0s 61ms/step - loss: 479.1511 - val_loss: 653.9788
Epoch 160/500
2/2 [==============================] - 0s 58ms/step - loss: 471.6626 - val_loss: 646.0137
Epoch 161/500
2/2 [==============================] - 0s 66ms/step - loss: 464.8938 - val_loss: 638.2812
Epoch 162/500
2/2 [==============================] - 0s 46ms/step - loss: 459.1080 - val_loss: 630.5826
Epoch 163/500
2/2 [==============================] - 0s 54ms/step - loss: 452.0908 - val_loss: 623.0438
Epoch 164/500
2/2 [==============================] - 0s 49ms/step - loss: 446.5945 - val_loss: 615.6225
Epoch 165/500
2/2 [==============================] - 0s 49ms/step - loss: 439.8385 - val_loss: 608.6822
Epoch 166/500
2/2 [==============================] - 0s 44ms/step - loss: 434.

Epoch 236/500
2/2 [==============================] - 0s 46ms/step - loss: 277.7043 - val_loss: 396.3391
Epoch 237/500
2/2 [==============================] - 0s 48ms/step - loss: 276.8195 - val_loss: 395.5530
Epoch 238/500
2/2 [==============================] - 0s 43ms/step - loss: 276.4000 - val_loss: 394.8390
Epoch 239/500
2/2 [==============================] - 0s 46ms/step - loss: 276.1345 - val_loss: 394.2739
Epoch 240/500
2/2 [==============================] - 0s 51ms/step - loss: 275.4991 - val_loss: 393.9583
Epoch 241/500
2/2 [==============================] - 0s 64ms/step - loss: 275.1370 - val_loss: 393.7559
Epoch 242/500
2/2 [==============================] - 0s 48ms/step - loss: 274.7949 - val_loss: 393.5365
Epoch 243/500
2/2 [==============================] - 0s 65ms/step - loss: 274.4803 - val_loss: 392.7431
Epoch 244/500
2/2 [==============================] - 0s 46ms/step - loss: 274.0540 - val_loss: 391.7751
Epoch 245/500
2/2 [==============================] - 0s 49ms/ste

Epoch 315/500
2/2 [==============================] - 0s 65ms/step - loss: 261.7049 - val_loss: 371.4809
Epoch 316/500
2/2 [==============================] - 0s 66ms/step - loss: 262.9341 - val_loss: 370.9186
Epoch 317/500
2/2 [==============================] - 0s 51ms/step - loss: 262.3671 - val_loss: 372.3909
Epoch 318/500
2/2 [==============================] - 0s 49ms/step - loss: 261.6978 - val_loss: 374.3589
Epoch 319/500
2/2 [==============================] - 0s 59ms/step - loss: 260.8101 - val_loss: 375.5389
Epoch 320/500
2/2 [==============================] - 0s 59ms/step - loss: 260.5002 - val_loss: 376.4287
Epoch 321/500
2/2 [==============================] - 0s 55ms/step - loss: 260.0381 - val_loss: 377.9738
Epoch 322/500
2/2 [==============================] - 0s 76ms/step - loss: 259.7726 - val_loss: 379.5163
Epoch 323/500
2/2 [==============================] - 0s 53ms/step - loss: 259.5808 - val_loss: 381.0111
Epoch 324/500
2/2 [==============================] - 0s 61ms/ste

Epoch 394/500
2/2 [==============================] - 0s 68ms/step - loss: 242.8928 - val_loss: 389.3855
Epoch 395/500
2/2 [==============================] - 0s 81ms/step - loss: 243.0513 - val_loss: 389.9371
Epoch 396/500
2/2 [==============================] - 0s 74ms/step - loss: 242.3871 - val_loss: 389.1611
Epoch 397/500
2/2 [==============================] - 0s 64ms/step - loss: 242.2126 - val_loss: 389.5205
Epoch 398/500
2/2 [==============================] - 0s 67ms/step - loss: 241.9023 - val_loss: 393.9361
Epoch 399/500
2/2 [==============================] - 0s 68ms/step - loss: 241.9055 - val_loss: 401.8156
Epoch 400/500
2/2 [==============================] - 0s 63ms/step - loss: 241.4684 - val_loss: 402.0117
Epoch 401/500
2/2 [==============================] - 0s 67ms/step - loss: 241.0802 - val_loss: 403.3374
Epoch 402/500
2/2 [==============================] - 0s 60ms/step - loss: 241.5274 - val_loss: 402.3848
Epoch 403/500
2/2 [==============================] - 0s 53ms/ste

Epoch 473/500
2/2 [==============================] - 0s 73ms/step - loss: 229.4336 - val_loss: 407.8979
Epoch 474/500
2/2 [==============================] - 0s 73ms/step - loss: 229.0516 - val_loss: 411.8314
Epoch 475/500
2/2 [==============================] - 0s 59ms/step - loss: 228.9993 - val_loss: 419.1356
Epoch 476/500
2/2 [==============================] - 0s 74ms/step - loss: 228.5110 - val_loss: 415.4926
Epoch 477/500
2/2 [==============================] - 0s 72ms/step - loss: 228.7409 - val_loss: 412.8724
Epoch 478/500
2/2 [==============================] - 0s 74ms/step - loss: 228.0468 - val_loss: 420.7188
Epoch 479/500
2/2 [==============================] - 0s 67ms/step - loss: 227.9216 - val_loss: 426.6577
Epoch 480/500
2/2 [==============================] - 0s 60ms/step - loss: 228.1456 - val_loss: 433.3272
Epoch 481/500
2/2 [==============================] - 0s 68ms/step - loss: 229.3875 - val_loss: 420.0409
Epoch 482/500
2/2 [==============================] - 0s 60ms/ste

## Testing Proposed Model w/TF with Rolling window of  4

In [12]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, LSTM, TimeDistributed, Activation, Dot, Concatenate
from tensorflow.keras.models import Model


# Spliting the data into training, validation, and test sets
train_data, test_data = train_test_split(data, test_size=0.2)
train_data, val_data = train_test_split(train_data, test_size=0.2)

# Define the encoder and decoder inputs
encoder_inputs = Input(shape=(None, 1))
decoder_inputs = Input(shape=(None, 1))

# Define the encoder LSTM layer
encoder_lstm = LSTM(180, return_sequences=True, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c]

# Define the decoder LSTM layer
decoder_lstm = LSTM(180, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)

# Adding the Attention mechanism 
attention = Dot(axes=[2, 2])([decoder_outputs, encoder_outputs])
attention = Activation('softmax')(attention)

context = Dot(axes=[2, 1])([attention, encoder_outputs])
decoder_combined_context = Concatenate(axis=-1)([context, decoder_outputs])

# Output layer
output = TimeDistributed(Dense(1, activation='linear'))(decoder_combined_context)

# Define the model
model = Model([encoder_inputs, decoder_inputs], output)
model.compile(optimizer='adam', loss='mse')

# Prepare data using the last 4 data points for the rolling window
window_size = 4

encoder_input_train = train_data.values[-window_size-1:-1, :, np.newaxis]
decoder_output_train = train_data.values[-window_size:, :, np.newaxis]
decoder_input_train = np.zeros_like(decoder_output_train)  # Initializing decoder inputs

# Teacher forcing: Setting up decoder inputs with true outputs from previous timestep
decoder_input_train[:-1] = decoder_output_train[:-1]

encoder_input_val = val_data.values[-window_size-1:-1, :, np.newaxis]
decoder_input_val = val_data.values[-window_size:, :, np.newaxis]
decoder_output_val = val_data.values[-window_size:, :, np.newaxis]

encoder_input_test = test_data.values[-window_size-1:-1, :, np.newaxis]
decoder_input_test = test_data.values[-window_size:, :, np.newaxis]
decoder_output_test = test_data.values[-window_size:, :, np.newaxis]

# Train the model using teacher forcing
model.fit(
    [encoder_input_train, decoder_input_train],
    decoder_output_train,
    validation_data=([encoder_input_val, decoder_input_val], decoder_output_val),
    epochs=500,
    batch_size=4
)

# Evaluate the model on the test set
preds = model.predict([encoder_input_test, decoder_input_test])
predictions = preds.squeeze()
test_values = decoder_output_test.squeeze()

mae = mean_absolute_error(test_values, predictions)
smape = 100 * np.mean(2 * np.abs(predictions - test_values) / (np.abs(predictions) + np.abs(test_values)))
rmse = np.sqrt(mean_squared_error(test_values, predictions))
r2 = r2_score(test_values, predictions)

print("MAE:", mae)
print("SMAPE:", smape)
print("RMSE:", rmse)
print("R2 Score:", r2)


Epoch 1/500
1/1 [==============================] - 4s 4s/step - loss: 15934.9189 - val_loss: 14873.7979
Epoch 2/500
1/1 [==============================] - 0s 45ms/step - loss: 15841.8291 - val_loss: 14780.8955
Epoch 3/500
1/1 [==============================] - 0s 43ms/step - loss: 15740.7812 - val_loss: 14694.2188
Epoch 4/500
1/1 [==============================] - 0s 44ms/step - loss: 15644.2168 - val_loss: 14614.3447
Epoch 5/500
1/1 [==============================] - 0s 42ms/step - loss: 15554.2227 - val_loss: 14536.2344
Epoch 6/500
1/1 [==============================] - 0s 54ms/step - loss: 15464.4648 - val_loss: 14457.9131
Epoch 7/500
1/1 [==============================] - 0s 49ms/step - loss: 15371.7793 - val_loss: 14377.6758
Epoch 8/500
1/1 [==============================] - 0s 48ms/step - loss: 15274.3770 - val_loss: 14295.2539
Epoch 9/500
1/1 [==============================] - 0s 47ms/step - loss: 15172.8604 - val_loss: 14209.9883
Epoch 10/500
1/1 [==============================

1/1 [==============================] - 0s 47ms/step - loss: 7437.1519 - val_loss: 6668.5679
Epoch 79/500
1/1 [==============================] - 0s 44ms/step - loss: 7359.2261 - val_loss: 6594.5317
Epoch 80/500
1/1 [==============================] - 0s 43ms/step - loss: 7282.7632 - val_loss: 6521.8877
Epoch 81/500
1/1 [==============================] - 0s 43ms/step - loss: 7208.0049 - val_loss: 6450.2656
Epoch 82/500
1/1 [==============================] - 0s 55ms/step - loss: 7135.1973 - val_loss: 6378.9771
Epoch 83/500
1/1 [==============================] - 0s 65ms/step - loss: 7063.0400 - val_loss: 6307.8110
Epoch 84/500
1/1 [==============================] - 0s 43ms/step - loss: 6990.8599 - val_loss: 6236.9844
Epoch 85/500
1/1 [==============================] - 0s 55ms/step - loss: 6918.9292 - val_loss: 6167.1167
Epoch 86/500
1/1 [==============================] - 0s 41ms/step - loss: 6847.6064 - val_loss: 6098.9487
Epoch 87/500
1/1 [==============================] - 0s 44ms/step - l

1/1 [==============================] - 0s 39ms/step - loss: 3340.6094 - val_loss: 2809.6763
Epoch 156/500
1/1 [==============================] - 0s 51ms/step - loss: 3305.9932 - val_loss: 2777.4202
Epoch 157/500
1/1 [==============================] - 0s 44ms/step - loss: 3271.5815 - val_loss: 2745.4526
Epoch 158/500
1/1 [==============================] - 0s 42ms/step - loss: 3237.5828 - val_loss: 2713.7766
Epoch 159/500
1/1 [==============================] - 0s 40ms/step - loss: 3203.9558 - val_loss: 2682.2983
Epoch 160/500
1/1 [==============================] - 0s 42ms/step - loss: 3170.5271 - val_loss: 2651.0774
Epoch 161/500
1/1 [==============================] - 0s 39ms/step - loss: 3137.2302 - val_loss: 2620.2588
Epoch 162/500
1/1 [==============================] - 0s 40ms/step - loss: 3104.2454 - val_loss: 2589.8938
Epoch 163/500
1/1 [==============================] - 0s 40ms/step - loss: 3071.6750 - val_loss: 2560.0530
Epoch 164/500
1/1 [==============================] - 0s 39ms

1/1 [==============================] - 0s 49ms/step - loss: 1532.5765 - val_loss: 1177.2695
Epoch 233/500
1/1 [==============================] - 0s 48ms/step - loss: 1517.8497 - val_loss: 1164.3849
Epoch 234/500
1/1 [==============================] - 0s 46ms/step - loss: 1503.2852 - val_loss: 1151.6530
Epoch 235/500
1/1 [==============================] - 0s 44ms/step - loss: 1488.8821 - val_loss: 1139.0717
Epoch 236/500
1/1 [==============================] - 0s 50ms/step - loss: 1474.6370 - val_loss: 1126.6409
Epoch 237/500
1/1 [==============================] - 0s 49ms/step - loss: 1460.5503 - val_loss: 1114.3578
Epoch 238/500
1/1 [==============================] - 0s 48ms/step - loss: 1446.6190 - val_loss: 1102.2214
Epoch 239/500
1/1 [==============================] - 0s 48ms/step - loss: 1432.8427 - val_loss: 1090.2294
Epoch 240/500
1/1 [==============================] - 0s 57ms/step - loss: 1419.2186 - val_loss: 1078.3815
Epoch 241/500
1/1 [==============================] - 0s 49ms

1/1 [==============================] - 0s 50ms/step - loss: 767.7245 - val_loss: 530.3880
Epoch 311/500
1/1 [==============================] - 0s 43ms/step - loss: 761.8067 - val_loss: 525.6806
Epoch 312/500
1/1 [==============================] - 0s 48ms/step - loss: 755.9625 - val_loss: 521.0395
Epoch 313/500
1/1 [==============================] - 0s 49ms/step - loss: 750.1906 - val_loss: 516.4642
Epoch 314/500
1/1 [==============================] - 0s 51ms/step - loss: 744.4907 - val_loss: 511.9542
Epoch 315/500
1/1 [==============================] - 0s 55ms/step - loss: 738.8621 - val_loss: 507.5081
Epoch 316/500
1/1 [==============================] - 0s 47ms/step - loss: 733.3036 - val_loss: 503.1258
Epoch 317/500
1/1 [==============================] - 0s 53ms/step - loss: 727.8147 - val_loss: 498.8063
Epoch 318/500
1/1 [==============================] - 0s 70ms/step - loss: 722.3947 - val_loss: 494.5488
Epoch 319/500
1/1 [==============================] - 0s 49ms/step - loss: 717.

Epoch 389/500
1/1 [==============================] - 0s 55ms/step - loss: 473.6893 - val_loss: 312.6779
Epoch 390/500
1/1 [==============================] - 0s 57ms/step - loss: 471.6320 - val_loss: 311.3488
Epoch 391/500
1/1 [==============================] - 0s 48ms/step - loss: 469.6069 - val_loss: 310.0486
Epoch 392/500
1/1 [==============================] - 0s 59ms/step - loss: 467.6086 - val_loss: 308.7776
Epoch 393/500
1/1 [==============================] - 0s 53ms/step - loss: 465.6368 - val_loss: 307.5353
Epoch 394/500
1/1 [==============================] - 0s 51ms/step - loss: 463.6918 - val_loss: 306.3204
Epoch 395/500
1/1 [==============================] - 0s 46ms/step - loss: 461.7740 - val_loss: 305.1316
Epoch 396/500
1/1 [==============================] - 0s 63ms/step - loss: 459.8845 - val_loss: 303.9662
Epoch 397/500
1/1 [==============================] - 0s 63ms/step - loss: 458.0233 - val_loss: 302.8224
Epoch 398/500
1/1 [==============================] - 0s 58ms/ste

Epoch 468/500
1/1 [==============================] - 0s 45ms/step - loss: 375.9215 - val_loss: 257.9461
Epoch 469/500
1/1 [==============================] - 0s 45ms/step - loss: 375.3168 - val_loss: 257.6595
Epoch 470/500
1/1 [==============================] - 0s 45ms/step - loss: 374.7319 - val_loss: 257.4067
Epoch 471/500
1/1 [==============================] - 0s 49ms/step - loss: 374.1426 - val_loss: 257.1749
Epoch 472/500
1/1 [==============================] - 0s 52ms/step - loss: 373.5352 - val_loss: 256.9620
Epoch 473/500
1/1 [==============================] - 0s 64ms/step - loss: 372.9120 - val_loss: 256.7737
Epoch 474/500
1/1 [==============================] - 0s 57ms/step - loss: 372.2874 - val_loss: 256.6145
Epoch 475/500
1/1 [==============================] - 0s 57ms/step - loss: 371.6770 - val_loss: 256.4811
Epoch 476/500
1/1 [==============================] - 0s 57ms/step - loss: 371.0895 - val_loss: 256.3634
Epoch 477/500
1/1 [==============================] - 0s 55ms/ste

## Testing Proposed Model w/TF with Rolling window of  3

In [13]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, LSTM, TimeDistributed, Activation, Dot, Concatenate
from tensorflow.keras.models import Model


# Spliting the data into training, validation, and test sets
train_data, test_data = train_test_split(data, test_size=0.2)
train_data, val_data = train_test_split(train_data, test_size=0.2)

# Define the encoder and decoder inputs
encoder_inputs = Input(shape=(None, 1))
decoder_inputs = Input(shape=(None, 1))

# Define the encoder LSTM layer
encoder_lstm = LSTM(180, return_sequences=True, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c]

# Define the decoder LSTM layer
decoder_lstm = LSTM(180, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)

# Adding the Attention mechanism 
attention = Dot(axes=[2, 2])([decoder_outputs, encoder_outputs])
attention = Activation('softmax')(attention)

context = Dot(axes=[2, 1])([attention, encoder_outputs])
decoder_combined_context = Concatenate(axis=-1)([context, decoder_outputs])

# Output layer
output = TimeDistributed(Dense(1, activation='linear'))(decoder_combined_context)

# Define the model
model = Model([encoder_inputs, decoder_inputs], output)
model.compile(optimizer='adam', loss='mse')

# Prepare data using the last 3 data points for the rolling window
window_size = 3

encoder_input_train = train_data.values[-window_size-1:-1, :, np.newaxis]
decoder_output_train = train_data.values[-window_size:, :, np.newaxis]
decoder_input_train = np.zeros_like(decoder_output_train)  # Initializing decoder inputs

# Teacher forcing: Setting up decoder inputs with true outputs from previous timestep
decoder_input_train[:-1] = decoder_output_train[:-1]

encoder_input_val = val_data.values[-window_size-1:-1, :, np.newaxis]
decoder_input_val = val_data.values[-window_size:, :, np.newaxis]
decoder_output_val = val_data.values[-window_size:, :, np.newaxis]

encoder_input_test = test_data.values[-window_size-1:-1, :, np.newaxis]
decoder_input_test = test_data.values[-window_size:, :, np.newaxis]
decoder_output_test = test_data.values[-window_size:, :, np.newaxis]

# Train the model using teacher forcing
model.fit(
    [encoder_input_train, decoder_input_train],
    decoder_output_train,
    validation_data=([encoder_input_val, decoder_input_val], decoder_output_val),
    epochs=500,
    batch_size=4
)

# Evaluate the model on the test set
preds = model.predict([encoder_input_test, decoder_input_test])
predictions = preds.squeeze()
test_values = decoder_output_test.squeeze()

mae = mean_absolute_error(test_values, predictions)
smape = 100 * np.mean(2 * np.abs(predictions - test_values) / (np.abs(predictions) + np.abs(test_values)))
rmse = np.sqrt(mean_squared_error(test_values, predictions))
r2 = r2_score(test_values, predictions)

print("MAE:", mae)
print("SMAPE:", smape)
print("RMSE:", rmse)
print("R2 Score:", r2)


Epoch 1/500
1/1 [==============================] - 4s 4s/step - loss: 15806.3291 - val_loss: 14562.9336
Epoch 2/500
1/1 [==============================] - 0s 42ms/step - loss: 15694.7939 - val_loss: 14447.2666
Epoch 3/500
1/1 [==============================] - 0s 43ms/step - loss: 15571.8252 - val_loss: 14341.2549
Epoch 4/500
1/1 [==============================] - 0s 39ms/step - loss: 15453.2764 - val_loss: 14253.6865
Epoch 5/500
1/1 [==============================] - 0s 42ms/step - loss: 15346.3096 - val_loss: 14166.6211
Epoch 6/500
1/1 [==============================] - 0s 42ms/step - loss: 15237.4678 - val_loss: 14077.9951
Epoch 7/500
1/1 [==============================] - 0s 46ms/step - loss: 15122.9717 - val_loss: 13989.4561
Epoch 8/500
1/1 [==============================] - 0s 43ms/step - loss: 15004.8779 - val_loss: 13901.9814
Epoch 9/500
1/1 [==============================] - 0s 43ms/step - loss: 14884.4180 - val_loss: 13815.4873
Epoch 10/500
1/1 [==============================

1/1 [==============================] - 0s 59ms/step - loss: 7373.4673 - val_loss: 6549.9160
Epoch 79/500
1/1 [==============================] - 0s 58ms/step - loss: 7305.6655 - val_loss: 6485.5215
Epoch 80/500
1/1 [==============================] - 0s 50ms/step - loss: 7238.7134 - val_loss: 6421.8882
Epoch 81/500
1/1 [==============================] - 0s 62ms/step - loss: 7172.3110 - val_loss: 6359.7603
Epoch 82/500
1/1 [==============================] - 0s 66ms/step - loss: 7107.0723 - val_loss: 6299.3267
Epoch 83/500
1/1 [==============================] - 0s 55ms/step - loss: 7043.6274 - val_loss: 6239.2642
Epoch 84/500
1/1 [==============================] - 0s 63ms/step - loss: 6981.1704 - val_loss: 6179.3833
Epoch 85/500
1/1 [==============================] - 0s 54ms/step - loss: 6919.3530 - val_loss: 6119.7856
Epoch 86/500
1/1 [==============================] - 0s 42ms/step - loss: 6857.8301 - val_loss: 6060.4624
Epoch 87/500
1/1 [==============================] - 0s 50ms/step - l

1/1 [==============================] - 0s 44ms/step - loss: 3502.4580 - val_loss: 2893.4954
Epoch 156/500
1/1 [==============================] - 0s 42ms/step - loss: 3469.7351 - val_loss: 2863.9329
Epoch 157/500
1/1 [==============================] - 0s 38ms/step - loss: 3437.4846 - val_loss: 2834.7334
Epoch 158/500
1/1 [==============================] - 0s 41ms/step - loss: 3405.6653 - val_loss: 2805.8977
Epoch 159/500
1/1 [==============================] - 0s 48ms/step - loss: 3374.2698 - val_loss: 2777.4407
Epoch 160/500
1/1 [==============================] - 0s 41ms/step - loss: 3343.2859 - val_loss: 2749.2996
Epoch 161/500
1/1 [==============================] - 0s 61ms/step - loss: 3312.6150 - val_loss: 2721.4668
Epoch 162/500
1/1 [==============================] - 0s 45ms/step - loss: 3282.2598 - val_loss: 2693.9397
Epoch 163/500
1/1 [==============================] - 0s 49ms/step - loss: 3252.2375 - val_loss: 2666.6826
Epoch 164/500
1/1 [==============================] - 0s 66ms

1/1 [==============================] - 0s 48ms/step - loss: 1734.1652 - val_loss: 1312.2557
Epoch 233/500
1/1 [==============================] - 0s 58ms/step - loss: 1719.0065 - val_loss: 1299.1056
Epoch 234/500
1/1 [==============================] - 0s 51ms/step - loss: 1704.0013 - val_loss: 1286.0999
Epoch 235/500
1/1 [==============================] - 0s 68ms/step - loss: 1689.1488 - val_loss: 1273.2374
Epoch 236/500
1/1 [==============================] - 0s 61ms/step - loss: 1674.4473 - val_loss: 1260.5173
Epoch 237/500
1/1 [==============================] - 0s 60ms/step - loss: 1659.8959 - val_loss: 1247.9381
Epoch 238/500
1/1 [==============================] - 0s 52ms/step - loss: 1645.4928 - val_loss: 1235.4983
Epoch 239/500
1/1 [==============================] - 0s 51ms/step - loss: 1631.2363 - val_loss: 1223.1964
Epoch 240/500
1/1 [==============================] - 0s 56ms/step - loss: 1617.1256 - val_loss: 1211.0312
Epoch 241/500
1/1 [==============================] - 0s 46ms

Epoch 310/500
1/1 [==============================] - 0s 48ms/step - loss: 923.4157 - val_loss: 633.0540
Epoch 311/500
1/1 [==============================] - 0s 42ms/step - loss: 916.8763 - val_loss: 627.8985
Epoch 312/500
1/1 [==============================] - 0s 50ms/step - loss: 910.4113 - val_loss: 622.8112
Epoch 313/500
1/1 [==============================] - 0s 55ms/step - loss: 904.0206 - val_loss: 617.7910
Epoch 314/500
1/1 [==============================] - 0s 42ms/step - loss: 897.7032 - val_loss: 612.8371
Epoch 315/500
1/1 [==============================] - 0s 57ms/step - loss: 891.4588 - val_loss: 607.9487
Epoch 316/500
1/1 [==============================] - 0s 44ms/step - loss: 885.2858 - val_loss: 603.1255
Epoch 317/500
1/1 [==============================] - 0s 46ms/step - loss: 879.1848 - val_loss: 598.3666
Epoch 318/500
1/1 [==============================] - 0s 52ms/step - loss: 873.1540 - val_loss: 593.6717
Epoch 319/500
1/1 [==============================] - 0s 45ms/ste

Epoch 389/500
1/1 [==============================] - 0s 47ms/step - loss: 587.7461 - val_loss: 386.4992
Epoch 390/500
1/1 [==============================] - 0s 55ms/step - loss: 585.2818 - val_loss: 384.9136
Epoch 391/500
1/1 [==============================] - 0s 57ms/step - loss: 582.8499 - val_loss: 383.3571
Epoch 392/500
1/1 [==============================] - 0s 46ms/step - loss: 580.4502 - val_loss: 381.8289
Epoch 393/500
1/1 [==============================] - 0s 57ms/step - loss: 578.0815 - val_loss: 380.3287
Epoch 394/500
1/1 [==============================] - 0s 74ms/step - loss: 575.7434 - val_loss: 378.8559
Epoch 395/500
1/1 [==============================] - 0s 71ms/step - loss: 573.4355 - val_loss: 377.4102
Epoch 396/500
1/1 [==============================] - 0s 65ms/step - loss: 571.1573 - val_loss: 375.9910
Epoch 397/500
1/1 [==============================] - 0s 45ms/step - loss: 568.9089 - val_loss: 374.5973
Epoch 398/500
1/1 [==============================] - 0s 63ms/ste

Epoch 468/500
1/1 [==============================] - 0s 42ms/step - loss: 466.5299 - val_loss: 319.8954
Epoch 469/500
1/1 [==============================] - 0s 56ms/step - loss: 465.6795 - val_loss: 319.4981
Epoch 470/500
1/1 [==============================] - 0s 49ms/step - loss: 464.8388 - val_loss: 319.2220
Epoch 471/500
1/1 [==============================] - 0s 53ms/step - loss: 464.0061 - val_loss: 318.9714
Epoch 472/500
1/1 [==============================] - 0s 58ms/step - loss: 463.1957 - val_loss: 318.5959
Epoch 473/500
1/1 [==============================] - 0s 58ms/step - loss: 462.3910 - val_loss: 318.3307
Epoch 474/500
1/1 [==============================] - 0s 45ms/step - loss: 461.5975 - val_loss: 318.1115
Epoch 475/500
1/1 [==============================] - 0s 39ms/step - loss: 460.8247 - val_loss: 317.7721
Epoch 476/500
1/1 [==============================] - 0s 44ms/step - loss: 460.0552 - val_loss: 317.5199
Epoch 477/500
1/1 [==============================] - 0s 55ms/ste

## Testing Proposed Model w/TF with Rolling window of  2

In [14]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, LSTM, TimeDistributed, Activation, Dot, Concatenate
from tensorflow.keras.models import Model


# Spliting the data into training, validation, and test sets
train_data, test_data = train_test_split(data, test_size=0.2)
train_data, val_data = train_test_split(train_data, test_size=0.2)

# Define the encoder and decoder inputs
encoder_inputs = Input(shape=(None, 1))
decoder_inputs = Input(shape=(None, 1))

# Define the encoder LSTM layer
encoder_lstm = LSTM(180, return_sequences=True, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c]

# Define the decoder LSTM layer
decoder_lstm = LSTM(180, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)

# Adding the Attention mechanism 
attention = Dot(axes=[2, 2])([decoder_outputs, encoder_outputs])
attention = Activation('softmax')(attention)

context = Dot(axes=[2, 1])([attention, encoder_outputs])
decoder_combined_context = Concatenate(axis=-1)([context, decoder_outputs])

# Output layer
output = TimeDistributed(Dense(1, activation='linear'))(decoder_combined_context)

# Define the model
model = Model([encoder_inputs, decoder_inputs], output)
model.compile(optimizer='adam', loss='mse')

# Prepare data using the last 2 data points for the rolling window
window_size = 2

encoder_input_train = train_data.values[-window_size-1:-1, :, np.newaxis]
decoder_output_train = train_data.values[-window_size:, :, np.newaxis]
decoder_input_train = np.zeros_like(decoder_output_train)  # Initializing decoder inputs

# Teacher forcing: Setting up decoder inputs with true outputs from previous timestep
decoder_input_train[:-1] = decoder_output_train[:-1]

encoder_input_val = val_data.values[-window_size-1:-1, :, np.newaxis]
decoder_input_val = val_data.values[-window_size:, :, np.newaxis]
decoder_output_val = val_data.values[-window_size:, :, np.newaxis]

encoder_input_test = test_data.values[-window_size-1:-1, :, np.newaxis]
decoder_input_test = test_data.values[-window_size:, :, np.newaxis]
decoder_output_test = test_data.values[-window_size:, :, np.newaxis]

# Train the model using teacher forcing
model.fit(
    [encoder_input_train, decoder_input_train],
    decoder_output_train,
    validation_data=([encoder_input_val, decoder_input_val], decoder_output_val),
    epochs=500,
    batch_size=4
)

# Evaluate the model on the test set
preds = model.predict([encoder_input_test, decoder_input_test])
predictions = preds.squeeze()
test_values = decoder_output_test.squeeze()

mae = mean_absolute_error(test_values, predictions)
smape = 100 * np.mean(2 * np.abs(predictions - test_values) / (np.abs(predictions) + np.abs(test_values)))
rmse = np.sqrt(mean_squared_error(test_values, predictions))
r2 = r2_score(test_values, predictions)

print("MAE:", mae)
print("SMAPE:", smape)
print("RMSE:", rmse)
print("R2 Score:", r2)


Epoch 1/500
1/1 [==============================] - 4s 4s/step - loss: 15281.1309 - val_loss: 11947.8613
Epoch 2/500
1/1 [==============================] - 0s 44ms/step - loss: 15138.0000 - val_loss: 11839.3652
Epoch 3/500
1/1 [==============================] - 0s 45ms/step - loss: 14993.6143 - val_loss: 11743.8018
Epoch 4/500
1/1 [==============================] - 0s 47ms/step - loss: 14861.5049 - val_loss: 11639.2441
Epoch 5/500
1/1 [==============================] - 0s 45ms/step - loss: 14716.9736 - val_loss: 11548.7217
Epoch 6/500
1/1 [==============================] - 0s 46ms/step - loss: 14581.9082 - val_loss: 11469.2061
Epoch 7/500
1/1 [==============================] - 0s 45ms/step - loss: 14450.5664 - val_loss: 11392.8965
Epoch 8/500
1/1 [==============================] - 0s 47ms/step - loss: 14316.3818 - val_loss: 11317.9727
Epoch 9/500
1/1 [==============================] - 0s 46ms/step - loss: 14181.4648 - val_loss: 11244.3867
Epoch 10/500
1/1 [==============================

1/1 [==============================] - 0s 59ms/step - loss: 6937.4263 - val_loss: 4760.4062
Epoch 79/500
1/1 [==============================] - 0s 60ms/step - loss: 6867.7964 - val_loss: 4700.8540
Epoch 80/500
1/1 [==============================] - 0s 48ms/step - loss: 6799.4761 - val_loss: 4642.5298
Epoch 81/500
1/1 [==============================] - 0s 54ms/step - loss: 6732.7388 - val_loss: 4585.4863
Epoch 82/500
1/1 [==============================] - 0s 48ms/step - loss: 6667.8604 - val_loss: 4529.8188
Epoch 83/500
1/1 [==============================] - 0s 58ms/step - loss: 6604.5752 - val_loss: 4475.6396
Epoch 84/500
1/1 [==============================] - 0s 63ms/step - loss: 6542.5020 - val_loss: 4422.7090
Epoch 85/500
1/1 [==============================] - 0s 61ms/step - loss: 6481.1533 - val_loss: 4370.2759
Epoch 86/500
1/1 [==============================] - 0s 65ms/step - loss: 6419.8442 - val_loss: 4317.8755
Epoch 87/500
1/1 [==============================] - 0s 69ms/step - l

1/1 [==============================] - 0s 48ms/step - loss: 3337.6733 - val_loss: 1814.0853
Epoch 156/500
1/1 [==============================] - 0s 41ms/step - loss: 3306.5266 - val_loss: 1790.8715
Epoch 157/500
1/1 [==============================] - 0s 45ms/step - loss: 3275.6655 - val_loss: 1767.8885
Epoch 158/500
1/1 [==============================] - 0s 56ms/step - loss: 3245.0637 - val_loss: 1745.1202
Epoch 159/500
1/1 [==============================] - 0s 55ms/step - loss: 3214.6968 - val_loss: 1722.5543
Epoch 160/500
1/1 [==============================] - 0s 42ms/step - loss: 3184.5457 - val_loss: 1700.1836
Epoch 161/500
1/1 [==============================] - 0s 41ms/step - loss: 3154.5981 - val_loss: 1678.0027
Epoch 162/500
1/1 [==============================] - 0s 46ms/step - loss: 3124.8506 - val_loss: 1656.0131
Epoch 163/500
1/1 [==============================] - 0s 54ms/step - loss: 3095.3054 - val_loss: 1634.3466
Epoch 164/500
1/1 [==============================] - 0s 41ms

1/1 [==============================] - 0s 47ms/step - loss: 1614.9858 - val_loss: 618.9450
Epoch 233/500
1/1 [==============================] - 0s 48ms/step - loss: 1600.2893 - val_loss: 609.9479
Epoch 234/500
1/1 [==============================] - 0s 50ms/step - loss: 1585.7443 - val_loss: 601.0771
Epoch 235/500
1/1 [==============================] - 0s 46ms/step - loss: 1571.3508 - val_loss: 592.3314
Epoch 236/500
1/1 [==============================] - 0s 49ms/step - loss: 1557.1078 - val_loss: 583.7090
Epoch 237/500
1/1 [==============================] - 0s 41ms/step - loss: 1543.0128 - val_loss: 575.2090
Epoch 238/500
1/1 [==============================] - 0s 52ms/step - loss: 1529.0653 - val_loss: 566.8295
Epoch 239/500
1/1 [==============================] - 0s 47ms/step - loss: 1515.2626 - val_loss: 558.5696
Epoch 240/500
1/1 [==============================] - 0s 48ms/step - loss: 1501.6042 - val_loss: 550.4274
Epoch 241/500
1/1 [==============================] - 0s 56ms/step - l

1/1 [==============================] - 0s 39ms/step - loss: 834.9582 - val_loss: 210.8553
Epoch 311/500
1/1 [==============================] - 0s 50ms/step - loss: 828.7340 - val_loss: 208.5249
Epoch 312/500
1/1 [==============================] - 0s 55ms/step - loss: 822.5811 - val_loss: 206.2463
Epoch 313/500
1/1 [==============================] - 0s 62ms/step - loss: 816.4986 - val_loss: 204.0187
Epoch 314/500
1/1 [==============================] - 0s 49ms/step - loss: 810.4841 - val_loss: 201.8410
Epoch 315/500
1/1 [==============================] - 0s 42ms/step - loss: 804.5339 - val_loss: 199.7117
Epoch 316/500
1/1 [==============================] - 0s 42ms/step - loss: 798.6449 - val_loss: 197.6296
Epoch 317/500
1/1 [==============================] - 0s 43ms/step - loss: 792.8163 - val_loss: 195.5931
Epoch 318/500
1/1 [==============================] - 0s 39ms/step - loss: 787.0504 - val_loss: 193.6008
Epoch 319/500
1/1 [==============================] - 0s 38ms/step - loss: 781.

Epoch 389/500
1/1 [==============================] - 0s 50ms/step - loss: 518.2970 - val_loss: 144.2735
Epoch 390/500
1/1 [==============================] - 0s 39ms/step - loss: 516.0060 - val_loss: 144.4584
Epoch 391/500
1/1 [==============================] - 0s 67ms/step - loss: 513.7460 - val_loss: 144.6586
Epoch 392/500
1/1 [==============================] - 0s 83ms/step - loss: 511.5157 - val_loss: 144.8730
Epoch 393/500
1/1 [==============================] - 0s 51ms/step - loss: 509.3160 - val_loss: 145.1005
Epoch 394/500
1/1 [==============================] - 0s 58ms/step - loss: 507.1457 - val_loss: 145.3414
Epoch 395/500
1/1 [==============================] - 0s 56ms/step - loss: 505.0045 - val_loss: 145.5957
Epoch 396/500
1/1 [==============================] - 0s 59ms/step - loss: 502.8919 - val_loss: 145.8625
Epoch 397/500
1/1 [==============================] - 0s 60ms/step - loss: 500.8079 - val_loss: 146.1409
Epoch 398/500
1/1 [==============================] - 0s 62ms/ste

Epoch 468/500
1/1 [==============================] - 0s 42ms/step - loss: 406.9650 - val_loss: 181.0709
Epoch 469/500
1/1 [==============================] - 0s 44ms/step - loss: 406.2150 - val_loss: 181.7730
Epoch 470/500
1/1 [==============================] - 0s 47ms/step - loss: 405.4766 - val_loss: 182.4727
Epoch 471/500
1/1 [==============================] - 0s 48ms/step - loss: 404.7495 - val_loss: 183.1671
Epoch 472/500
1/1 [==============================] - 0s 45ms/step - loss: 404.0334 - val_loss: 183.8541
Epoch 473/500
1/1 [==============================] - 0s 43ms/step - loss: 403.3280 - val_loss: 184.5332
Epoch 474/500
1/1 [==============================] - 0s 46ms/step - loss: 402.6331 - val_loss: 185.2048
Epoch 475/500
1/1 [==============================] - 0s 50ms/step - loss: 401.9486 - val_loss: 185.8702
Epoch 476/500
1/1 [==============================] - 0s 44ms/step - loss: 401.2744 - val_loss: 186.5319
Epoch 477/500
1/1 [==============================] - 0s 65ms/ste

## Testing Proposed Model w/TF with Rolling window of  1

In [15]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, LSTM, TimeDistributed, Activation, Dot, Concatenate
from tensorflow.keras.models import Model


# Spliting the data into training, validation, and test sets
train_data, test_data = train_test_split(data, test_size=0.2)
train_data, val_data = train_test_split(train_data, test_size=0.2)

# Define the encoder and decoder inputs
encoder_inputs = Input(shape=(None, 1))
decoder_inputs = Input(shape=(None, 1))

# Define the encoder LSTM layer
encoder_lstm = LSTM(180, return_sequences=True, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c]

# Define the decoder LSTM layer
decoder_lstm = LSTM(180, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)

# Adding the Attention mechanism 
attention = Dot(axes=[2, 2])([decoder_outputs, encoder_outputs])
attention = Activation('softmax')(attention)

context = Dot(axes=[2, 1])([attention, encoder_outputs])
decoder_combined_context = Concatenate(axis=-1)([context, decoder_outputs])

# Output layer
output = TimeDistributed(Dense(1, activation='linear'))(decoder_combined_context)

# Define the model
model = Model([encoder_inputs, decoder_inputs], output)
model.compile(optimizer='adam', loss='mse')

# Prepare data using the last 2 data points for the rolling window
window_size = 1

encoder_input_train = train_data.values[-window_size-1:-1, :, np.newaxis]
decoder_output_train = train_data.values[-window_size:, :, np.newaxis]
decoder_input_train = np.zeros_like(decoder_output_train)  # Initializing decoder inputs

# Teacher forcing: Setting up decoder inputs with true outputs from previous timestep
decoder_input_train[:-1] = decoder_output_train[:-1]

encoder_input_val = val_data.values[-window_size-1:-1, :, np.newaxis]
decoder_input_val = val_data.values[-window_size:, :, np.newaxis]
decoder_output_val = val_data.values[-window_size:, :, np.newaxis]

encoder_input_test = test_data.values[-window_size-1:-1, :, np.newaxis]
decoder_input_test = test_data.values[-window_size:, :, np.newaxis]
decoder_output_test = test_data.values[-window_size:, :, np.newaxis]

# Train the model using teacher forcing
model.fit(
    [encoder_input_train, decoder_input_train],
    decoder_output_train,
    validation_data=([encoder_input_val, decoder_input_val], decoder_output_val),
    epochs=500,
    batch_size=4
)

# Evaluate the model on the test set
preds = model.predict([encoder_input_test, decoder_input_test])
predictions = preds.squeeze()
test_values = decoder_output_test.squeeze()

mae = mean_absolute_error(test_values, predictions)
smape = 100 * np.mean(2 * np.abs(predictions - test_values) / (np.abs(predictions) + np.abs(test_values)))
rmse = np.sqrt(mean_squared_error(test_values, predictions))
r2 = r2_score(test_values, predictions)

print("MAE:", mae)
print("SMAPE:", smape)
print("RMSE:", rmse)
print("R2 Score:", r2)


Epoch 1/500
1/1 [==============================] - 5s 5s/step - loss: 15640.3926 - val_loss: 16102.9785
Epoch 2/500
1/1 [==============================] - 0s 45ms/step - loss: 15497.3525 - val_loss: 16036.3965
Epoch 3/500
1/1 [==============================] - 0s 40ms/step - loss: 15351.5049 - val_loss: 15969.9775
Epoch 4/500
1/1 [==============================] - 0s 40ms/step - loss: 15200.1787 - val_loss: 15903.1338
Epoch 5/500
1/1 [==============================] - 0s 41ms/step - loss: 15039.9434 - val_loss: 15837.0615
Epoch 6/500
1/1 [==============================] - 0s 39ms/step - loss: 14869.7900 - val_loss: 15769.1104
Epoch 7/500
1/1 [==============================] - 0s 39ms/step - loss: 14690.4307 - val_loss: 15701.0039
Epoch 8/500
1/1 [==============================] - 0s 39ms/step - loss: 14507.1318 - val_loss: 15633.0498
Epoch 9/500
1/1 [==============================] - 0s 40ms/step - loss: 14326.1289 - val_loss: 15564.6631
Epoch 10/500
1/1 [==============================

1/1 [==============================] - 0s 38ms/step - loss: 7117.0137 - val_loss: 11029.6133
Epoch 79/500
1/1 [==============================] - 0s 38ms/step - loss: 7055.7197 - val_loss: 10975.2100
Epoch 80/500
1/1 [==============================] - 0s 40ms/step - loss: 6994.5781 - val_loss: 10917.0645
Epoch 81/500
1/1 [==============================] - 0s 40ms/step - loss: 6933.9937 - val_loss: 10854.2676
Epoch 82/500
1/1 [==============================] - 0s 40ms/step - loss: 6875.0171 - val_loss: 10789.2979
Epoch 83/500
1/1 [==============================] - 0s 37ms/step - loss: 6817.1040 - val_loss: 10724.8877
Epoch 84/500
1/1 [==============================] - 0s 38ms/step - loss: 6760.1821 - val_loss: 10662.0830
Epoch 85/500
1/1 [==============================] - 0s 37ms/step - loss: 6703.4541 - val_loss: 10601.9863
Epoch 86/500
1/1 [==============================] - 0s 42ms/step - loss: 6646.7876 - val_loss: 10545.0479
Epoch 87/500
1/1 [==============================] - 0s 39ms

1/1 [==============================] - 0s 52ms/step - loss: 3607.2551 - val_loss: 7792.6714
Epoch 156/500
1/1 [==============================] - 0s 44ms/step - loss: 3574.7451 - val_loss: 7754.5957
Epoch 157/500
1/1 [==============================] - 0s 52ms/step - loss: 3542.3491 - val_loss: 7717.2822
Epoch 158/500
1/1 [==============================] - 0s 46ms/step - loss: 3510.2498 - val_loss: 7680.7749
Epoch 159/500
1/1 [==============================] - 0s 35ms/step - loss: 3478.4438 - val_loss: 7644.9897
Epoch 160/500
1/1 [==============================] - 0s 35ms/step - loss: 3446.6956 - val_loss: 7609.8247
Epoch 161/500
1/1 [==============================] - 0s 41ms/step - loss: 3414.9429 - val_loss: 7575.1206
Epoch 162/500
1/1 [==============================] - 0s 41ms/step - loss: 3383.3901 - val_loss: 7540.5889
Epoch 163/500
1/1 [==============================] - 0s 50ms/step - loss: 3352.2693 - val_loss: 7505.8862
Epoch 164/500
1/1 [==============================] - 0s 50ms

1/1 [==============================] - 0s 40ms/step - loss: 1804.2751 - val_loss: 5699.1094
Epoch 233/500
1/1 [==============================] - 0s 36ms/step - loss: 1788.4421 - val_loss: 5678.4790
Epoch 234/500
1/1 [==============================] - 0s 37ms/step - loss: 1772.7612 - val_loss: 5658.0020
Epoch 235/500
1/1 [==============================] - 0s 36ms/step - loss: 1757.2318 - val_loss: 5637.7651
Epoch 236/500
1/1 [==============================] - 0s 36ms/step - loss: 1741.8511 - val_loss: 5617.5396
Epoch 237/500
1/1 [==============================] - 0s 41ms/step - loss: 1726.6167 - val_loss: 5597.4380
Epoch 238/500
1/1 [==============================] - 0s 49ms/step - loss: 1711.5302 - val_loss: 5577.6270
Epoch 239/500
1/1 [==============================] - 0s 46ms/step - loss: 1696.5876 - val_loss: 5557.7559
Epoch 240/500
1/1 [==============================] - 0s 37ms/step - loss: 1681.7870 - val_loss: 5537.8750
Epoch 241/500
1/1 [==============================] - 0s 38ms

1/1 [==============================] - 0s 37ms/step - loss: 932.5964 - val_loss: 4376.6240
Epoch 310/500
1/1 [==============================] - 0s 39ms/step - loss: 925.3125 - val_loss: 4363.1050
Epoch 311/500
1/1 [==============================] - 0s 40ms/step - loss: 918.1123 - val_loss: 4349.7793
Epoch 312/500
1/1 [==============================] - 0s 42ms/step - loss: 910.9949 - val_loss: 4336.6494
Epoch 313/500
1/1 [==============================] - 0s 49ms/step - loss: 903.9606 - val_loss: 4323.7212
Epoch 314/500
1/1 [==============================] - 0s 57ms/step - loss: 897.0070 - val_loss: 4311.0034
Epoch 315/500
1/1 [==============================] - 0s 55ms/step - loss: 890.1344 - val_loss: 4298.4990
Epoch 316/500
1/1 [==============================] - 0s 65ms/step - loss: 883.3406 - val_loss: 4286.2046
Epoch 317/500
1/1 [==============================] - 0s 66ms/step - loss: 876.6243 - val_loss: 4274.1079
Epoch 318/500
1/1 [==============================] - 0s 73ms/step - l

1/1 [==============================] - 0s 40ms/step - loss: 560.8880 - val_loss: 3586.6641
Epoch 388/500
1/1 [==============================] - 0s 38ms/step - loss: 558.0870 - val_loss: 3579.0376
Epoch 389/500
1/1 [==============================] - 0s 44ms/step - loss: 555.3221 - val_loss: 3571.4917
Epoch 390/500
1/1 [==============================] - 0s 38ms/step - loss: 552.5933 - val_loss: 3564.0234
Epoch 391/500
1/1 [==============================] - 0s 38ms/step - loss: 549.8996 - val_loss: 3556.6101
Epoch 392/500
1/1 [==============================] - 0s 41ms/step - loss: 547.2411 - val_loss: 3549.2522
Epoch 393/500
1/1 [==============================] - 0s 45ms/step - loss: 544.6174 - val_loss: 3541.9548
Epoch 394/500
1/1 [==============================] - 0s 47ms/step - loss: 542.0278 - val_loss: 3534.7078
Epoch 395/500
1/1 [==============================] - 0s 43ms/step - loss: 539.4721 - val_loss: 3527.5107
Epoch 396/500
1/1 [==============================] - 0s 40ms/step - l

1/1 [==============================] - 0s 63ms/step - loss: 425.1264 - val_loss: 3130.2996
Epoch 466/500
1/1 [==============================] - 0s 51ms/step - loss: 424.1810 - val_loss: 3125.9172
Epoch 467/500
1/1 [==============================] - 0s 67ms/step - loss: 423.2495 - val_loss: 3121.5867
Epoch 468/500
1/1 [==============================] - 0s 73ms/step - loss: 422.3319 - val_loss: 3117.3044
Epoch 469/500
1/1 [==============================] - 0s 70ms/step - loss: 421.4277 - val_loss: 3113.0557
Epoch 470/500
1/1 [==============================] - 0s 72ms/step - loss: 420.5371 - val_loss: 3108.8232
Epoch 471/500
1/1 [==============================] - 0s 62ms/step - loss: 419.6595 - val_loss: 3104.6096
Epoch 472/500
1/1 [==============================] - 0s 59ms/step - loss: 418.7949 - val_loss: 3100.4319
Epoch 473/500
1/1 [==============================] - 0s 78ms/step - loss: 417.9434 - val_loss: 3096.2998
Epoch 474/500
1/1 [==============================] - 0s 79ms/step - l